LSTM trained on gridded forcings for each station

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate
import torch.autograd as autograd

In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda' if USE_CUDA else 'cpu')
torch.manual_seed(0)
np.random.seed(0)

writer = SummaryWriter()

CUDA Available


In [3]:
station_data_dict = load_data.load_train_test_lstm()
data_runoff = load_data.load_discharge_gr4j_vic()

../src/load_data.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(os.path.join(dir, f), skiprows=2, skipfooter=1, index_col=False, header=None, names=['runoff'], na_values='-1.2345')
/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
class LSTMRegression(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, batch_size, dropout):
            super(LSTMRegression, self).__init__()
            self.batch_size = batch_size
            self.hidden_dim = hidden_dim
            self.num_layers = num_layers
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout)
            self.linear = nn.Linear(hidden_dim, 1)
            self.hidden = self.init_hidden()
        def init_hidden(self):
            return (torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device),
                    torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device))

        def forward(self, input):
            lstm_out, self.hidden = self.lstm(input, self.hidden)
            return self.linear(lstm_out[-1])

In [5]:
list(station_data_dict.keys())[10:]

['02GC026',
 '02GD004',
 '02GE007',
 '02GG002',
 '02GG003',
 '02GG006',
 '02GG009',
 '02GG013',
 '04159492',
 '04159900',
 '04160600',
 '04161820',
 '04164000',
 '04165500',
 '04166100',
 '04166500',
 '04174500',
 '04176500',
 '04177000',
 '04193500',
 '04195820',
 '04196800',
 '04197100',
 '04198000',
 '04199000',
 '04199500',
 '04200500',
 '04207200',
 '04208504',
 '04209000',
 '04212100',
 '04213000',
 '04213500',
 '04214500',
 '04215000',
 '04215500']

In [6]:
predictions = {}
actuals = {}
models = {}
seq_len = 5 * 24
train_start = datetime.strptime('2012-01-01', '%Y-%m-%d') + timedelta(days=seq_len // 24 + 1)
train_ends = ['2012-12-31', '2013-02-28', '2013-04-30', '2013-06-30', '2013-08-31', '2013-10-31']
test_ends = train_ends[1:] + ['2013-12-31']


plot_list = ['04159492']
median_nse_list = []
for cv_iter in range(len(train_ends)):
    train_end = train_ends[cv_iter]
    test_start = datetime.strptime(train_end, '%Y-%m-%d') + timedelta(days=1)
    test_end = test_ends[cv_iter]
    print('Train: {} - {}, Test: {} - {}'.format(train_start.strftime('%Y-%m-%d'), train_end, test_start.strftime('%Y-%m-%d'), test_end))
    
    nse_list = []
    cv_name = '_CV_{}-{}'.format(test_start.strftime('%Y-%m-%d'), test_end)
    for station in list(station_data_dict.keys())[:10]:
        station_rdrs = station_data_dict[station]
        station_runoff = data_runoff[data_runoff['station'] == station].set_index('date')
        if any(station_runoff['runoff'].isna()):
            print('Station', station, 'had NA runoff values. Skipping.')
            continue

        station_train = station_rdrs.loc[train_start : train_end]
        station_test = station_rdrs.loc[test_start : test_end]
        num_train_days = len(pd.date_range(train_start, train_end, freq='D'))

        x = np.zeros((seq_len, len(pd.date_range(train_start, test_end, freq='D')), station_rdrs.shape[1]))
        for day in range(x.shape[1]):
            x[:,day,:] = station_rdrs[train_start - timedelta(hours = seq_len - 1) + timedelta(days=day) : train_start + timedelta(days=day)]

        # Scale training data
        scalers = []  # save scalers to apply them to test data later
        x_train = x[:,:num_train_days,:]
        for i in range(x.shape[2]):
            scalers.append(preprocessing.StandardScaler())
            x_train[:,:,i] = scalers[i].fit_transform(x_train[:,:,i].reshape((-1, 1))).reshape(x_train[:,:,i].shape)
        x_train = torch.from_numpy(x_train).float().to(device)
        y_train = torch.from_numpy(station_runoff.loc[train_start:train_end, 'runoff'].to_numpy()).float().to(device)

        # Train model
        learning_rate = 2e-3
        patience = 50
        min_improvement = 0.05
        best_loss_model = (-1, np.inf, None)

        # Prepare model
        H = 200
        batch_size = 3
        lstm_layers = 2
        dropout = 0.3
        model = LSTMRegression(station_rdrs.shape[1], H, lstm_layers, batch_size, dropout).to(device)
        loss_fn = torch.nn.MSELoss(reduction='mean')
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        for epoch in range(300):
            epoch_losses = []
            for i in range(num_train_days // batch_size):
                model.hidden = model.init_hidden()
                y_pred = model(x_train[:,i*batch_size : (i+1)*batch_size,:])

                loss = loss_fn(y_pred, y_train[i*batch_size : (i+1)*batch_size].reshape((batch_size,1))).to(device)
                epoch_losses.append(loss.item())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            epoch_loss = np.array(epoch_losses).mean()
            writer.add_scalar('loss_' + station + cv_name, epoch_loss, epoch)
            if epoch_loss < best_loss_model[1] - min_improvement:
                best_loss_model = (epoch, epoch_loss, model.state_dict())  # new best model
            elif epoch > best_loss_model[0] + patience:
                print('Patience exhausted in epoch {}. Best loss was {}'.format(epoch, best_loss_model[1]))
                break

        print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
        model.load_state_dict(best_loss_model[2])
        model.eval()        

        # scale test data
        x_test = x[:,num_train_days:,:]
        for i in range(x.shape[2]):
            x_test[:,:,i] = scalers[i].transform(x_test[:,:,i].reshape((-1, 1))).reshape(x_test[:,:,i].shape)
        # if batch size doesn't align with number of samples, add dummies to the last batch
        if x_test.shape[1] % batch_size != 0:
            x_test = np.concatenate([x_test, np.zeros((x_test.shape[0], batch_size - (x_test.shape[1] % batch_size), x_test.shape[2]))], axis=1)

        x_test = torch.from_numpy(x_test).float().to(device)
        predict = station_runoff[test_start:test_end].copy()
        predict['runoff'] = np.nan
        pred_array = np.array([])
        for i in range(x_test.shape[1] // batch_size):
            pred_array = np.concatenate([pred_array, model(x_test[:,i*batch_size : (i+1)*batch_size,:]).detach().cpu().numpy().reshape(batch_size)])
        predict['runoff'] = pred_array[:predict.shape[0]]  # ignore dummies
        predictions[station] = predict
        actuals[station] = station_runoff['runoff'].loc[test_start:test_end]
        models[station] = model
        
        nse = evaluate.evaluate_daily(station + cv_name, predict['runoff'], actuals[station], writer=writer)
        nse_list.append(nse)
    
    print('  NSEs: {}:'.format(nse_list))
    writer.add_histogram('NSE', np.array(nse_list), cv_iter)
    median_nse_list.append(np.median(nse_list))

Train: 2012-01-07 - 2012-12-31, Test: 2013-01-01 - 2013-02-28
Patience exhausted in epoch 171. Best loss was 0.17365201848442666
Using best model from epoch 120 which had loss 0.17365201848442666


/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


Patience exhausted in epoch 208. Best loss was 0.09215442987697316
Using best model from epoch 157 which had loss 0.09215442987697316
Patience exhausted in epoch 163. Best loss was 0.10070072326683051
Using best model from epoch 112 which had loss 0.10070072326683051
Patience exhausted in epoch 156. Best loss was 0.22471882685980138
Using best model from epoch 105 which had loss 0.22471882685980138
Patience exhausted in epoch 251. Best loss was 5.482428623487552
Using best model from epoch 200 which had loss 5.482428623487552
Patience exhausted in epoch 170. Best loss was 0.041212958513066646
Using best model from epoch 119 which had loss 0.041212958513066646
Patience exhausted in epoch 160. Best loss was 0.09987562822619414
Using best model from epoch 109 which had loss 0.09987562822619414
Patience exhausted in epoch 127. Best loss was 0.12189095471815865
Using best model from epoch 76 which had loss 0.12189095471815865
Patience exhausted in epoch 174. Best loss was 0.0752233746762309

In [7]:
median_nse_list

[0.2776757499108159,
 -0.17724980381846822,
 -0.4286692368085874,
 -0.14102652817979333,
 -0.13638789181295463,
 -0.00395837975089286]

In [8]:
writer.close()